In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
labels = df.to_numpy()[:, 0]
scaler = StandardScaler()

df = df.drop(["label"], axis = 1)
dataset = df.to_numpy()

VARIANCE = 0.98

pca = PCA(VARIANCE)

scaled_dataset = scaler.fit_transform(dataset)
pca.fit(scaled_dataset)
scaled_dataset = pca.transform(scaled_dataset)

start = time.perf_counter()
kmeans = MiniBatchKMeans(init='k-means++', n_init = 10, n_clusters = 6000).fit(scaled_dataset)
end = time.perf_counter()
print("Done in ", end - start, "seconds...")

In [26]:
import numpy as np 

print(kmeans.labels_)
df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
labels = df.to_numpy()[:, 0]


dic = {}
for i in range(len(kmeans.labels_)):
    val = kmeans.labels_[i]
    if val in dic:
        dic[val][labels[i]] += 1
    else:
        dic[val] = [0 for i in range(26)]
        dic[val][labels[i]] += 1

for key in dic:
    dic[key] = np.argmax(dic[key])

print(dic)




In [27]:
import matplotlib.pyplot as plt 

test_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
test = test_df.to_numpy()

labels_2 = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
                      
def show_img(img_idx):
    plt.imshow(test[img_idx][1:].reshape(28, 28), cmap='gray')

def get_label(img_idx):
    return labels_2[test[img_idx][0]]

print(get_label(1))
show_img(1)

In [34]:

scaled_test = scaler.fit_transform(test)
scaled_test = pca.transform(scaled_test[:, 1:])

predictions_int = kmeans.predict(scaled_test)

predictions = []
for i in range(len(predictions_int)):
    if predictions_int[i] in dic:
        predictions.append(dic[predictions_int[i]])
    else:
        predictions.append(0)

num_correct = 0
for i in range(test.shape[0]):
    if test[i][0] == predictions[i]:
        num_correct += 1
        
print("Accuracy is ", num_correct/test.shape[0] * 100, "%")





In [ ]:
"""Example"""
img_idx = 10

show_img(img_idx)
prediction = kmeans.predict([scaled_test[img_idx, :]])
print(labels[dic[prediction[0]]])